# Bruhin, Fehr, and Schunk, 2019, "Many Faces of Human Sociality: Uncovering the Distribution and Stability of Social Preferences", Table 2

#### Authors:  

- Massimiliano Pozzi (Bocconi University, pozzi.massimiliano@studbocconi.it)
- Salvatore Nunnari (Bocconi University, salvatore.nunnari@unibocconi.it)

#### Description: 

The code in this Jupyter notebook performs the finite mixture estimates with three types to replicate Table 2

This notebook was tested with the following packages versions:
- Pozzi:   (Anaconda 4.10.3 on Windows 10 Pro) : python 3.8.3, numpy 1.18.5, pandas 1.0.5, scipy 1.5.0, numdifftools 0.9.40
- Nunnari: (Anaconda 4.10.1 on macOS 10.15.7): python 3.8.10, numpy 1.20.2, pandas 1.2.4, scipy 1.6.2, numdifftools 0.9.39

In [150]:
# Import the necessary libraries

import numpy as np
from scipy.stats import norm  
import pandas as pd
import scipy.optimize as opt
from numdifftools import Gradient, Hessian

## 1. Data Cleaning and Data Preparation

We import the relevant datasets containing the data on the 39 dictator games and 78 reciprocity games in Session 1 and Session 2. As the authors, we remove from these datasets those subjects that behaved very inconsistenly throughout the games. These subjects are identified from the individual estimates that we do not run in this notebook. The file "dropped_subjects_section4paragraph2.csv" contains the IDs of these subjects. We also import the aggregate estimates that we obtained in the notebook to replicate Table 1. We will use these estimates as a starting point for the minimization algorithm.

In [2]:
# Import the three datasets and drop the inconsistent subjects

dt1 = pd.read_csv('../input/choices_exp1.csv') # import data on session 1
dt2 = pd.read_csv('../input/choices_exp2.csv') # import data on session 2

# import data with ID of subjects to drop
data_drop = pd.read_csv('../input/dropped_subjects_section4paragraph2.csv', usecols=[0], names=['sid'], header=None) 

dt1=dt1[~dt1.sid.isin(data_drop.sid)] # drop the session 1 subjects whose IDs are listed in the data_drop dataframe (14 individuals)
dt2=dt2[~dt2.sid.isin(data_drop.sid)] # drop the session 2 subjects whose IDs are listed in the data_drop dataframe (14 individuals)

# sort the dataframe by sid so that the first 117 observations are for individual one, the next 117 for individual two etc

dt1.sort_values(by=['sid'],inplace=True)
dt2.sort_values(by=['sid'],inplace=True)

# import the aggregate estimates for session 1 and session 2 saved in the table1_python.csv in the output folder

res_s1 = pd.read_csv('../output/table1_python.csv', usecols=["estimates_s1"]).values.flatten() # load aggregate estimates for session 1
res_s1 = [*res_s1[0:4], np.log(res_s1[4])]                                                     # we need the log of sigma

res_s2 = pd.read_csv('../output/table1_python.csv', usecols=["estimates_s2"]).values.flatten() # load aggregate estimates for session 1
res_s2 = [*res_s2[0:4], np.log(res_s2[4])]                                                     # we need the log of sigma

We now create indicators for allocation x or allocation y: indicators_x is made by the columns s_x, r_x, q, v of the dataframe dt1, indicators_y is made by the columns s_y, r_y, q, v.

In [3]:
# Create indicators_x and indicators_y. These are the columns s, r, q, v

indicators_x = np.column_stack((dt1.s_x,dt1.r_x,dt1.q,dt1.v))
indicators_y = np.column_stack((dt1.s_y,dt1.r_y,dt1.q,dt1.v))

## 2. Define the Model and the Likelihood (Sections 2 and 3 in the Paper)

### General Model

A generic player A's utility is given by:

$$ U^A = (1-\alpha s -\beta r - \gamma q - \delta v)⋅\Pi^A + (\alpha s +\beta r + \gamma q + \delta v)⋅\Pi^B $$ 

where &Pi;<sup>A</sup> represents player A's payoff, &Pi;<sup>B</sup> represents player B's payoff, s=1 if &Pi;<sup>A</sup>&lt; &Pi;<sup>B</sup> and 0 otherwise, r=1 if &Pi;<sup>A</sup>&gt; &Pi;<sup>B</sup> and 0 otherwise. For example &alpha; < 0  implies that the subject is behindness averse, while a value of &beta; > 0 implies that the subject is aheadness averse; q is an indicator that takes value one if B behaved kindly (capturing positive reciprocity), while v is an indicator that takes value one if B behaved unkindly (capturing negative reciprocity).

The authors model heterogeneity with a Random Utility Model, so that individual A prefers allocation X to allocation Y if and only if: 

$$ U^A(X_g;\theta)+\epsilon_{X_g} \geq U^A(Y_g;\theta)+\epsilon_{Y_g} $$ 

where X<sub>g</sub> is the allocation (&Pi;<sup>A</sup><sub>Xg</sub>, &Pi;<sup>A</sup><sub>Yg</sub>, r<sub>Xg</sub>, s<sub>Xg</sub>, q<sub>Xg</sub>, v<sub>Xg</sub>) in game g; &theta; is a vector containing the parameters &alpha;, &beta;, &gamma;, &delta;; and &epsilon;<sub>Xg</sub> is a random noise term which follows a type-1 extreme value distribution with the reciprocal of &sigma; as scale parameter. 

Thus, the probability that individual A chooses allocation X in game g is given by: 

$$ Pr(C_g= X_g ; \theta, \sigma, X_g, Y_g)=Pr(U^A(X_g;\theta)- U^A(Y_g;\theta) \geq \epsilon_{Y_g}-\epsilon_{X_g})=\frac{exp(\sigma U^A(X_g;\theta))}{exp(\sigma U^A(X_g;\theta))+exp(\sigma U^A(Y_g;\theta))} $$

We can then write subject is contribution to the total likelihood of observing the data given the parameters as follows: 

$$ f(\theta,\sigma;X,Y,C_i)=\prod_{g=1}^{G} Pr(C_g= X_g ; \theta, \sigma, X_g, Y_g)^{I(C_{ig}=X_g)}⋅Pr(C_g= Y_g ; \theta, \sigma, X_g, Y_g)^{1-I(C_{ig}=X_g)} $$

where G is the number of games each individual plays (117) and I is an indicator function that is equal to one if player A chooses allocation X.

### Finite Mixture Model

A finite mixture model assumes that the population is made up of a finite number of types K, each characterized by its own set of parameters (&theta;<sub>k</sub>, &sigma;<sub>k</sub>). A given subject i's likeliood contribution depends now on the whole set of parameters and on the vector containing the types' shares in the population &Psi; = (&theta;<sub>1</sub>, ..., &theta;<sub>K</sub>, &sigma;<sub>1</sub>, ..., &sigma;<sub>K</sub>, &pi;<sub>1</sub>, ..., &pi;<sub>K-1</sub>). The individual likelihood contribution can be then written as:

$$ L(\Psi; X, Y, C_i) = \sum_{k=1}^K \pi_kf(\theta_k,\sigma_k;X,Y,C_i) $$

Our goal is to minimize the negative of the logarithm of the sum of the individual likelihood contribution written above using the EM-algorithm. Below, we explain each step of the algoritm, but for a more rigorous discussion please refer to Dempster, A.P, N. M. Laird and D. B. Rubin, 1977, ["Maximum Likelihood from Incomplete Data via the EM Algorithm"](http://snunnari.github.io/dempster.pdf), *Journal of the Royal Statistical Society, Series B (Methodological)*, 39(1): 1&ndash;38 and to McLachlan, Geoffrey, Sharon Lee, and Suren Rathnayake, 2019, ["Finite Mixture Models"](http://snunnari.github.io/mclachlan.pdf), *Annual Review of Statistics and Its Application*, 6: 355&ndash;378.


In [151]:
# Define the component density function. This would be f(θ_k, σ_k; X, Y, C_i) above for a specific set of parameters (θ_k, σ_k).
# v is the vector of parameters (θ_k, σ_k)
# y is the choice of the player
# self_x is the payoff when choosing x (left)
# self_y is the payoff when choosing y (right)
# indicators are the ones explained above
# This function returns a 1x160 vector with the likelihood contribution for each individual (nr. of unique individual is 160)

def singlecomp(v,y,self_x, other_x, self_y, other_y, indicators_x,indicators_y):

    beta = v[0:-1]              # parameter vector θ. a 1x4 vector
    sigma = np.exp(v[-1])       # choice sensitivity σ. We take the exp since it can only take positive values
    
    lli = indicators_x @ beta   # matrix product. we obtain a 1x18720 vector. Each element is (αs+βr+γq+δv) for the single game
    rli = indicators_y @ beta
    uleft  = sigma*( (1-lli) * self_x + lli * other_x) # utility when choosing allocation X (left)
    uright = sigma*( (1-rli) * self_y + rli * other_y) # utility when choosing allocation Y (right)
    
    # probs is a 1x18720 vector containing the likelihood of each observation in the data (a single game for a generic player A)
    
    probs  = np.array((np.exp(uleft)/(np.exp(uleft)+np.exp(uright)))**y * (np.exp(uright)/(np.exp(uleft)+np.exp(uright)))**(1-y)) 

    # indprob is a 117x160 matrix. each column contains the likelihood of the observations for a single individual
    
    indprob = np.reshape(probs, (160, 117)) 

    # 1x160 vector containing the likelihood contribution for a single individual. We took the product of a column

    indlike = np.prod(indprob, axis=1) 
    
    # in theory it should not happen that a probability is equal to zero, but for some combinations of parameters it does. If prob = 0 we set it equal to a small value 

    index_p0 = [i for i in range(0,len(indlike)) if indlike[i]==0.0]

    for i in index_p0:
        indlike[i] = 1E-8
    
    return indlike 

In [152]:
# This function returns a 160x3 matrix. Column one contains the individual likelihood contribution using the parameters for type one.
# Column two contains the individual likelihood contribution for type two etc. Using the notation from above a generic element i in the first column 
# is what we called f(θ_1, σ_1; X, Y, C_i). Remember that we assumed there are three types (K=3).
# vall is a 1x15 vector that contains the parameters (θ,σ) for component 1,2 and 3, what we called (θ_1,σ_1,θ_2,σ_2,θ_3,σ_3).


def compdens(vall,y,self_x, other_x,self_y,other_y,indicators_x,indicators_y):

    theta1 = vall[0:5]    # parameters for first component  α1, β1, γ1, δ1, σ1. Careful [0:5] takes only five elements (from position 0 to 4)
    theta2 = vall[5:10]   # parameters for second component α2, β2, γ2, δ2, σ2
    theta3 = vall[10:15]  # parameters for third component  α3, β3, γ3, δ3, σ3

    comp1 = singlecomp(theta1,y,self_x, other_x, self_y,other_y,indicators_x,indicators_y) #1x160 vector containing f(θ_1, σ_1; X, Y, C_i) for i=1,...,160

    comp2 = singlecomp(theta2,y,self_x, other_x, self_y,other_y,indicators_x,indicators_y)

    comp3 = singlecomp(theta3,y,self_x, other_x, self_y,other_y,indicators_x,indicators_y)

    comp = np.column_stack((comp1,comp2,comp3)) # 160x3 matrix explained above

    return comp

We now define the ex-post probability that an individual i belongs to type k. In the code for the E-step function, we denote with tau.

$$ \tau_{ik}= \frac{\hat{\pi}_kf(\hat{\theta}_k,\hat{\sigma}_k;X,Y,C_i)}{\sum_{m=1}^K \hat{\pi}_mf(\hat{\theta}_m,\hat{\sigma}_m;X,Y,C_i) } $$

The E-step function returns the following 160x3 matrix:

$$  \tau= \begin{bmatrix} 
	\tau_{11} & \tau_{12} & \tau_{13} \\
	\tau_{21} & \tau_{22} & \tau_{23} \\
	... & ... & ... \\
    \tau_{1601} & \tau_{1602} & \tau_{1603} \\
	\end{bmatrix} $$

In [153]:
# This function returns a 160x3 matrix containing the posterior probability of an individual to be of type one (first column), two (second column) and three (third column).
# Element in the first row first column contains the ex-post probability of individual 1 being of type 1, element in the first row second column
# contains the ex-post probability of individual 1 being of type 2 etc.
# mix is a 1x3 vector containing the shares of type one, two and three in the population, what we called (π_1, π_2, π_3)

def estep(vall,y,self_x, other_x, self_y,other_y,indicators_x,indicators_y,mix):

    comp = compdens(vall,y,self_x, other_x,self_y,other_y,indicators_x,indicators_y)

    # m is a 160x3 matrix. first colunm is the weight for the first component, second column the weight for the second component etc

    m = ((np.ones((160,3))) * mix)

    numerator = m * comp # 160x3 matrix. each element is π_k f(θ_k,σ_k;X,Y,C_i)

    denominator = ((np.ones((160,3))).T * np.sum(numerator,axis=1)).T # 160x3 matrix. elements in the same rows are equal. They are the denominator for individual 1 up to 160

    # tau is our 160x3 matrix of ex-post probabilities for individual i belonging to type k

    tau = (np.ones((160, 3)) * (numerator / denominator))
    
    return tau

In [154]:
# This function returns the negative of the sum of the individual observation log likelihood weighted by the ex-post probabilities for individuak i to belong to type k
# This is the function to minimize in the M-step

def maxloglike(vall,y,self_x, other_x, self_y,other_y,indicators_x,indicators_y, tau):

    comp = compdens(vall,y,self_x, other_x,self_y,other_y,indicators_x,indicators_y)

    mll = -np.sum(tau * np.log(comp))

    return mll 

In [155]:
# Define the M-step function. This function minimizes maxloglike and returns two vectors.
# mix is a 1x3 vector containing the shares of the types in the population, namely (π_1, π_2, π_3) that will be used in the E-step
# vall_res is a 1x15 vector containing the new sets of parameters (θ_1,σ_1,θ_2,σ_2,θ_3,σ_3) that will be used in the E-step

def mstep(vall,y,self_x, other_x, self_y,other_y,indicators_x,indicators_y, tau):

    sol = opt.minimize(maxloglike,vall,
                   args=(y,self_x, other_x, self_y,other_y, indicators_x, indicators_y, tau),
                   method='nelder-mead')
    
    mix = np.mean(tau, axis=0)
    vall_res = sol.x
    
    return mix, vall_res, tau

In [156]:
# This function returns the finite mixture negative log likelihood we want to minimize. This is the sum over individuals of the negative log of what we called L(Ψ;X,Y,C_i)
# psi is a 1x17 vector containing the 5 parameters for each type (5x3) and the two mixtures. It's what we defined as Ψ.

def totallike(psi,y,self_x, other_x, self_y,other_y,indicators_x,indicators_y):

    vall = psi[0:15]              # 1x15 vector containing parameters for each type
    mix  = psi[15:17]             # here mix is a 1x2 vector containing π_1 and π_2
    mix  = [*mix, 1-np.sum(mix)]  # here mix contains also π_3 = 1 - (π_1 + π_2)
    
    comp = compdens(vall,y,self_x, other_x,self_y,other_y,indicators_x,indicators_y)
    m = ((np.ones((160,3))) * mix)

    tll = -sum(np.log(np.sum(m * comp, axis=1)))

    return tll

In [157]:
# The function estimates runs the EM-algoritm and returns a series of objects.
# theta_v is the 1x15 vector containing the estimates on the parameters for each type (θ_1,σ_1, θ_2,σ_2, θ_3,σ_3)
# mix is a 1x3 vector containing the types' shares (π_1, π_2, π_3)
# the third object returned is the value of the log likelihood obtained
# tau is a 160x3 matrix of ex-post probabilities
# psi is a 1x17 vector containing theta_v and the first two mixtures

def estimate(y, self_x, other_x, self_y, other_y, indicators_x, indicators_y, maxiter, diffcrit):

    # Starting conditions. random starting guesses close to the pooled ones. dimension is 1x15. 5 parameters for each type preferences

    theta_v = [*res_s1,*res_s1,*res_s1] * np.random.uniform(0.8,1.2,15) 

    # Starting guesses for the mixtures

    m0  = np.random.uniform(0.5,2.0,3)
    mix = m0/np.sum(m0)

    # Initial EM control flow parameters

    iteration = 1   # nr iterations
    diff  = 1       # difference between likelihoods in two different steps (llnew - llold)
    llold = 0       # old log likelihood
    llnew = 1       # new log likelihood

    while (iteration < maxiter+1 and np.abs(diff) > diffcrit):

        print("EM type maximization: ", iteration, "/", maxiter)
        print("Working on E-step.")
    
        tau_em = estep(theta_v,y,self_x, other_x, self_y,other_y,indicators_x,indicators_y,mix)

        print("Working on M-step.")

        m_step = mstep(theta_v,y,self_x, other_x, self_y,other_y,indicators_x,indicators_y, tau_em)

        mix = m_step[0]
        theta_v = m_step[1]
        psi = [*theta_v,*mix[0:2]]

        # Adjust EM control flow parameters

        iteration =  iteration + 1
        llold = llnew
        llnew = totallike(psi,y,self_x, other_x, self_y,other_y,indicators_x,indicators_y)
        diff = llnew - llold

        print("Achieved Log Likelihood:", -np.round(llnew,4))

    print("End of EM-steps")

    # Maximize the likelihood directly.

    sol = opt.minimize(totallike,psi,
                   args=(y,self_x, other_x, self_y,other_y, indicators_x, indicators_y),
                   method='nelder-Mead')

    print("Achieved final Log Likelihood:", -np.round(sol.fun,4))
    
    psi = sol.x
    theta_v = psi[0:15]
    mix = psi[15:17]
    mix = [*mix,1-np.sum(mix)]

    # Retrieve the taus

    tau = estep(theta_v,y,self_x, other_x, self_y,other_y,indicators_x,indicators_y,mix)
    
    return theta_v, mix, -sol.fun, tau, psi

## 3. Estimation

### Point Estimates

We now estimate the model using the EM-algorithm. We print the value of the log likelihood achieved in each step. 20 iterations are sufficient for convergence. On our machines, the computations take around 10 minutes, depending on the random starting points and their distance to the argmin vector.

In [158]:
# This cell computes the estimates. This takes around 10 minutes on our machines

args=(dt1['choice_x'],dt1['self_x'],dt1['other_x'],dt1['self_y'],dt1['other_y'],indicators_x,indicators_y)
solmixture = estimate(*args,20,5e-5)

EM type maximization:  1 / 20
Working on E-step.
Working on M-step.
Achieved Log Likelihood: -5017.7065
EM type maximization:  2 / 20
Working on E-step.
Working on M-step.
Achieved Log Likelihood: -4791.959
EM type maximization:  3 / 20
Working on E-step.
Working on M-step.
Achieved Log Likelihood: -4544.4075
EM type maximization:  4 / 20
Working on E-step.
Working on M-step.
Achieved Log Likelihood: -4381.0295
EM type maximization:  5 / 20
Working on E-step.
Working on M-step.
Achieved Log Likelihood: -4268.0563
EM type maximization:  6 / 20
Working on E-step.
Working on M-step.
Achieved Log Likelihood: -4219.2943
EM type maximization:  7 / 20
Working on E-step.
Working on M-step.
Achieved Log Likelihood: -4207.4715
EM type maximization:  8 / 20
Working on E-step.
Working on M-step.
Achieved Log Likelihood: -4203.7581
EM type maximization:  9 / 20
Working on E-step.
Working on M-step.
Achieved Log Likelihood: -4203.1938
EM type maximization:  10 / 20
Working on E-step.
Working on M-st

In [159]:
# the results we want

theta_v = solmixture[0]           # parameters for the three types
mixture = solmixture[1]           # the mixture probabilities
likelihood = solmixture[2]        # value of the likelihood
posterior_prob = solmixture[3]    # individual posterior prob. of being of a certain type
psi = solmixture[4]               # vector [theta_v,mixture]

results_sm1 = np.round([*psi[0:4],np.exp(psi[4]),*psi[5:9],np.exp(psi[9]),*psi[10:14],np.exp(psi[14]),*mixture], 3) # our vector with the results, rounded up to the third decimal 

### Standard Errors

We now estimate individual cluster robust standard errors. These are computed by taking the square root of the diagonal elements of the following matrix: 

$$ Adj⋅(H^{-1} @ G @ H^{-1}) $$ 

Where Adj is an adjustment for the degree of freedoms and the number of clusters:

$$ Adj = \frac{Nr.observations-1}{Nr.observations-Nr.parameters}⋅\frac{Nr.clusters}{Nr.clusters-1} $$ 

H<sup>-1</sup> is the inverse of the hessian of the negative log-likelihood evaluated in the minimum (our estimates). @ stands for matrix multiplication and G is a 5x5 matrix of gradient contributions. Call the gradient of the log likelihood function for a generic individual i in the following way:

$$  g_i(y|\theta) = [log f_i(y|\theta)]' = \frac{\partial}{\partial \theta} log f_i(y|\theta) $$

Where &theta; is the parameters vector and f<sub>i</sub>(y|&theta;) the likelihood function. Then G is defined as follows:

$$ G = \sum_j \left[\sum_{i \in c_j}g_i(y|\hat{\theta})\right]^T\left[\sum_{i \in c_j}g_i(y|\hat{\theta})\right] $$

where J is the number of clusters (the number of unique individuals) and c<sub>j</sub> is a generic cluster j, that includes all observations for a specific individual. For more information on how to compute standard errors when using maximum likelihood, we refer the reader to David A. Freedman, 2006, ["On The So-Called 'Huber Sandwich Estimator' and 'Robust Standard Errors'"](https://snunnari.github.io/freedman.pdf), *The American Statistician*, 60:4, 299-302).

In [160]:
# We now define a series of auxiliary functions that are necessary to compute the gradient contribution G. In this notebook we avoid to use analytical derivatives
# to be as general as possible, so that the code can be applied to any problem

# This function takes as input a dataset and a scalar and returns a dataframe containing only observations for individual whose ID is equal to that scalar

def getind(dataset,sid):
    dataset_ind = dataset[dataset.sid == sid]
    return dataset_ind

# This function is equivalent to singlecomp but takes as input only the observations for an individual i, so it returns a scalar

def ind_singlecomp(v,y,self_x, other_x, self_y,other_y,indicators_x,indicators_y):

    beta = v[0:-1]              # parameter vector θ. a 1x4 vector
    sigma = np.exp(v[-1])       # choice sensitivity σ. We take the exp since it can only take positive values
    
    lli = indicators_x @ beta 
    rli = indicators_y @ beta
    uleft  = sigma*( (1-lli) * self_x + lli * other_x) # utility when choosing allocation X (left)
    uright = sigma*( (1-rli) * self_y + rli * other_y) # utility when choosing allocation Y (right)
    
    # probs is a 1x117 vector containing the likelihood of each observation for individual i in the data
    
    probs  = np.array((np.exp(uleft)/(np.exp(uleft)+np.exp(uright)))**y * (np.exp(uright)/(np.exp(uleft)+np.exp(uright)))**(1-y)) 

    # indlike is a scalar containing the likelihood contribution of an individual for a single compoment. We called this f(θ_k,σ_k;X,Y,C_i)
    
    indlike = np.prod(probs) 
    
    # in theory it should not happen that a probability is equal to zero, but for some combinations of parameters it does. If prob = 0 we set it equal to a small value 
    
    if indlike==0.0: indlike=1e-8
    
    return indlike 

# This function is equivalnet to compdens but takes as input only the observations for an individual i, so it returns a 1x3 vector

def ind_compdens(vall,y,self_x, other_x,self_y,other_y,indicators_x,indicators_y):
    
    theta1 = vall[0:5]    # parameters for first component  α1, β1, γ1, δ1, σ1
    theta2 = vall[5:10]   # parameters for second component α2, β2, γ2, δ2, σ2
    theta3 = vall[10:15]  # parameters for third component  α3, β3, γ3, δ3, σ3

    comp1 = ind_singlecomp(theta1,y,self_x, other_x, self_y,other_y,indicators_x,indicators_y) # f(θ_1, σ_1; X, Y, C_i)
    comp2 = ind_singlecomp(theta2,y,self_x, other_x, self_y,other_y,indicators_x,indicators_y)
    comp3 = ind_singlecomp(theta3,y,self_x, other_x, self_y,other_y,indicators_x,indicators_y)

    # comp is a 1x3 vector. It contains f(θ_1, σ_1; X, Y, C_i), f(θ_2, σ_2; X, Y, C_i), f(θ_3, σ_3; X, Y, C_i)
    
    comp = [comp1,comp2,comp3] 

    return comp

# This function is equivalent to totallike but takes as input only the observations for an individual i. It returns the negative log likelihood for a single individual
# summing ind_tottallike over all individuals will return the same result as the function totallike

def ind_totallike(psi,y,self_x, other_x, self_y,other_y,indicators_x,indicators_y):

    vall = psi[0:15]              # 1x15 vector containing parameters for each type
    mix  = psi[15:18]             # here mix is a 1x3 vector containing π_1, π_2 and π_3
    
    comp = ind_compdens(vall,y,self_x, other_x,self_y,other_y,indicators_x,indicators_y)

    tll = -np.log(np.sum(mix * comp))

    return tll

# This function is equivalent to totallike but takes as parameters all three mixtures, even if we need only two of them. We do this so that the hessian includes also
# the effect of π_3.

def totallike_mixture(psi,y,self_x, other_x, self_y,other_y,indicators_x,indicators_y):

    vall = psi[0:15]              # 1x15 vector containing parameters for each type
    mix  = psi[15:18]             # here mix is a 1x3 vector containing π_1 and π_2
    
    comp = compdens(vall,y,self_x, other_x,self_y,other_y,indicators_x,indicators_y)
    m = ((np.ones((160,3))) * mix)

    tll = -sum(np.log(np.sum(m * comp, axis=1)))

    return tll

In [161]:
# Define the function that computes the matrix of individual gradient contribution G. We compute the gradient numerically using the numdifftools package

def gradcontr(dataset, parameters):
    G = 0.0
    for sid in np.unique(dataset.sid): # loop over the individuals IDs
        dt1_ind = getind(dataset,sid)   # dataframe for individual wid
        
        # args needed to compute the individual likelihood
        args_ind=(dt1_ind['choice_x'],dt1_ind['self_x'],dt1_ind['other_x'],dt1_ind['self_y'],dt1_ind['other_y'],
              np.column_stack((dt1_ind.s_x,dt1_ind.r_x,dt1_ind.q,dt1_ind.v)),
              np.column_stack((dt1_ind.s_y,dt1_ind.r_y,dt1_ind.q,dt1_ind.v)))
        
        gradcontr_ind = gradfun(parameters,*args_ind) # gradient of the negloglike function evaluated at the minimum
        
        G += np.outer(gradcontr_ind,gradcontr_ind) # G is the sum over individuals of the outer product of gradcontr_ind
        
    return G

In [163]:
# Compute the individual cluster robust standard errors

import warnings
warnings.filterwarnings('ignore') # This is to avoid showing RuntimeWarning regarding overflow when computing the hessian and the gradient. This warning does not affect the results.
                                  # We could avoid it by adding a check on the value of uleft/uright. We are using this command since we are sure of the results,
                                  # we do not suggest suppressing errors otherwise.

# Compute the hessian
Hfun = Hessian(totallike_mixture)
hessian = Hfun([*theta_v, *mixture], *args)      # hessian
hess_inv = np.linalg.inv(hessian)                # inverse of the hessian

# Compute the matrix of gradient contribution
gradfun = Gradient(ind_totallike)
grad_contribution = gradcontr(dt1, [*theta_v,*mixture])

# Compute the adjustment for degree of freedoms and number of clusters
adj = (len(dt1.sid)-1)/(len(dt1.sid)-len([*theta_v, *mixture])) * len(np.unique(dt1.sid))/(len(np.unique(dt1.sid))-1)

varcov_estimates = adj *(hess_inv @ grad_contribution @ hess_inv) # var-cov matrix of our estimates
se_sm1 = np.sqrt(np.diag((varcov_estimates)))                     # individual cluster robust standard errors

se_sm1 = [*se_sm1[0:4],np.sqrt(np.exp(psi[4])**2*se_sm1[4]**2),*se_sm1[5:9],np.sqrt(np.exp(psi[9])**2*se_sm1[9]**2),
          *se_sm1[10:14],np.sqrt(np.exp(psi[14])**2*se_sm1[14]**2),*se_sm1[15:18]]

In [164]:
# We do the same for session 2

# Create indicators for the second dataframe

indicators_x2 = np.column_stack((dt2.s_x,dt2.r_x,dt2.q,dt2.v))
indicators_y2 = np.column_stack((dt2.s_y,dt2.r_y,dt2.q,dt2.v))

# Compute the estimates

args2=(dt2['choice_x'],dt2['self_x'],dt2['other_x'],dt2['self_y'],dt2['other_y'],indicators_x2,indicators_y2)
solmixture2 = estimate(*args2,20,5e-5)

# the results we want

theta_v2 = solmixture2[0]           # parameters for the three types
mixture2 = solmixture2[1]           # the mixture probabilities
likelihood2 = solmixture2[2]        # value of the likelihood
posterior_prob2 = solmixture2[3]    # individual posterior prob. of being of a certain type
psi2 = solmixture2[4]               # vector [theta_v,mixture]

results_sm2 = np.round([*psi2[0:4],np.exp(psi2[4]),*psi2[5:9],np.exp(psi2[9]),*psi2[10:14],np.exp(psi2[14]),*mixture2], 3) # our vector with the results, rounded up to the third decimal 

EM type maximization:  1 / 20
Working on E-step.
Working on M-step.
Achieved Log Likelihood: -4049.0409
EM type maximization:  2 / 20
Working on E-step.
Working on M-step.
Achieved Log Likelihood: -3499.2295
EM type maximization:  3 / 20
Working on E-step.
Working on M-step.
Achieved Log Likelihood: -3277.0981
EM type maximization:  4 / 20
Working on E-step.
Working on M-step.
Achieved Log Likelihood: -3191.3322
EM type maximization:  5 / 20
Working on E-step.
Working on M-step.
Achieved Log Likelihood: -3169.4282
EM type maximization:  6 / 20
Working on E-step.
Working on M-step.
Achieved Log Likelihood: -3166.4129
EM type maximization:  7 / 20
Working on E-step.
Working on M-step.
Achieved Log Likelihood: -3166.3269
EM type maximization:  8 / 20
Working on E-step.
Working on M-step.
Achieved Log Likelihood: -3166.3206
EM type maximization:  9 / 20
Working on E-step.
Working on M-step.
Achieved Log Likelihood: -3166.3198
EM type maximization:  10 / 20
Working on E-step.
Working on M-s

In [165]:
# Compute the individual cluster robust standard errors

# Compute the hessian

hessian2 = Hfun([*theta_v2, *mixture2], *args2)      # hessian
hess_inv2 = np.linalg.inv(hessian2)                  # inverse of the hessian

# Compute the matrix of gradient contribution

grad_contribution2 = gradcontr(dt2, [*theta_v2, *mixture2])

# Compute the adjustment for degree of freedoms and number of clusters

adj = (len(dt2.sid)-1)/(len(dt2.sid)-len([*theta_v2, *mixture2])) * len(np.unique(dt2.sid))/(len(np.unique(dt2.sid))-1)

varcov_estimates2 = adj *(hess_inv2 @ grad_contribution2 @ hess_inv2) # var-cov matrix of our estimates
se_sm2 = np.sqrt(np.diag((varcov_estimates2)))                     # individual cluster robust standard errors

se_sm2 = [*se_sm2[0:4],np.sqrt(np.exp(psi2[4])**2*se_sm2[4]**2),*se_sm2[5:9],np.sqrt(np.exp(psi2[9])**2*se_sm2[9]**2),
          *se_sm2[10:14],np.sqrt(np.exp(psi2[14])**2*se_sm2[14]**2),*se_sm2[15:18]]

### Hypothesis Testing

We now do some hypothesis testing on the parameters we obtained. We compute the z-test statistics and the corresponding p-values to check if the parameters we obtained are statistically different from zero.

In [166]:
# Compute the z-test statistics and the corresponding p-values to check if the parameters are statistically different from zero

# Session 1. np.array since it supports element-wise operations

zvalues_s1 = np.array([*theta_v,*mixture])/np.array(se_sm1)
pvalues_s1 = 2*(1-norm.cdf(np.abs(zvalues_s1),0,1))

# Session 2

zvalues_s2 = np.array([*theta_v2,*mixture2])/np.array(se_sm2)
pvalues_s2 = 2*(1-norm.cdf(np.abs(zvalues_s2),0,1))

## 4. Print and Save Estimation Results

We print two tables: one for Session 1 and one for Session 2 with point estimates, individual cluster robust standard errors, and p-values for the hypothesis that the parameters are statistically different from zero. We then save the results as a single csv file in the output folder. This replicates Table 2 in the paper.

There are some discrepancies between the standard errors we obtain and those in Table 2 of the paper. The differences in the SEs for the point estimates of Session 1 are due to small differences in the point estimates themselves which are likely due to the use of a different minimization algorithm (we use Nelder-Mead, which is gradient-free, while the authors write down derivatives in closed form and use BFGS). The differences in the SEs for the mixture probabilities of Sessions 1 and 2 are probably due to the fact that our code takes a shortcut and computes the gradient of a subject's log likelihood instead of computing the sum of the gradients of each single choice's log likelihood for the same subject (which is the theoretically correct way of computing the SEs; see, for example, the discussion in the paper cited above). The reason for taking this shortcut is that the theoretically correct object is complex to compute, given the nature of the log likelihood function, and that the shortcut actually returns the same SEs as the authors' for the other parameters. Finally, we should note that, to compute hessians and gradients, we used finite differences rather than automatic differentiation. The reason is that the autograd python library (which we used for automatic differentiation in the notebook to replicate Table 1 of Augenblick and Rabin) does not work as intended for this complex model and the more recent jax library is not available on Windows. 

In [173]:
# print the results
# Note that if you run the code yourself the parameters will not be necessarily in this order: strongly altruistic, moderately altruistic,
# behindness averse. It depends on the starting conditions of the mixtures and on how the algorithm moves.

from IPython.display import display

param_names = ["π: Type's shares in the population",
               "α: Weight on other's payoff when behind",
               "β: Weight on other's payoff when ahead",
               "γ: Measure of positive reciprocity",
               "δ: Measure of negative reciprocity",
               "σ: Choice sensitivity"]

table21 = pd.DataFrame({"Parameters":param_names,"Str_altr_est":[results_sm1[16],*results_sm1[5:10]],"Str_altr_se":np.round([se_sm1[16],*se_sm1[5:10]],3),
                        "pval_str":np.round([pvalues_s1[16],*pvalues_s1[5:10]],3),"Mod_altr_est":[results_sm1[15],*results_sm1[0:5]],
                        "Mod_altr_se":np.round([se_sm1[15],*se_sm1[0:5]],3),"pval_mod":np.round([pvalues_s1[15],*pvalues_s1[0:5]],3),
                        "Beh_aver_est":[results_sm1[17],*results_sm1[10:15]],"Beh_aver_se":np.round([se_sm1[17],*se_sm1[10:15]],3),
                        "pval_beh":np.round([pvalues_s1[17],*pvalues_s1[10:15]],3)})

table22 = pd.DataFrame({"Parameters":param_names,"Str_altr_est":[results_sm2[15],*results_sm2[0:5]],"Str_altr_se":np.round([se_sm2[15],*se_sm2[0:5]],3),
                        "pval_str":np.round([pvalues_s2[15],*pvalues_s2[0:5]],3),"Mod_altr_est":[results_sm2[16],*results_sm2[5:10]],
                        "Mod_altr_se":np.round([se_sm2[16],*se_sm2[5:10]],3),"pval_mod":np.round([pvalues_s2[16],*pvalues_s2[5:10]],3),
                        "Beh_aver_est":[results_sm2[17],*results_sm2[10:15]],"Beh_aver_se":np.round([se_sm2[17],*se_sm2[10:15]],3),"pval_beh":np.round([pvalues_s2[17],*pvalues_s2[10:15]],3)})

print("Table 2. Finite mixture estimation (K=3) in session 1.")
print("The three types are strongly altruistic, moderately altruistic and behindness averse.")
display(table21)
print("Number of observations:","{:,}".format(len(dt1.sid)))
print("Number of subjects:","{:,}".format(len(np.unique(dt1.sid))))
print("Log likelihood:","{:,.2f}".format(likelihood))
print("")
print("Table 2. Finite mixture estimation (K=3) in session 2.")
display(table22)
print("Number of observations:","{:,}".format(len(dt2.sid)))
print("Number of subjects:","{:,}".format(len(np.unique(dt2.sid))))
print("Log likelihood:","{:,.2f}".format(likelihood2))

Table 2. Finite mixture estimation (K=3) in session 1.
The three types are strongly altruistic, moderately altruistic and behindness averse.


,Parameters,Str_altr_est,Str_altr_se,pval_str,Mod_altr_est,Mod_altr_se,pval_mod,Beh_aver_est,Beh_aver_se,pval_beh
0,π: Type's shares in the population,0.405,0.059,0.000,0.474,0.057,0.000,0.121,0.044,0.006
1,α: Weight on other's payoff when behind,0.159,0.040,0.000,0.065,0.013,0.000,-0.435,0.124,0.000
2,β: Weight on other's payoff when ahead,0.463,0.030,0.000,0.130,0.017,0.000,-0.145,0.159,0.364
3,γ: Measure of positive reciprocity,0.151,0.027,0.000,-0.001,0.012,0.938,0.171,0.127,0.179
4,δ: Measure of negative reciprocity,-0.054,0.027,0.049,-0.027,0.012,0.018,-0.075,0.180,0.676
5,σ: Choice sensitivity,0.018,0.001,0.000,0.032,0.002,0.000,0.008,0.002,0.000


Number of observations: 18,720
Number of subjects: 160
Log likelihood: -4,202.71

Table 2. Finite mixture estimation (K=3) in session 2.


,Parameters,Str_altr_est,Str_altr_se,pval_str,Mod_altr_est,Mod_altr_se,pval_mod,Beh_aver_est,Beh_aver_se,pval_beh
0,π: Type's shares in the population,0.356,0.048,0.0,0.544,0.059,0.000,0.100,0.025,0.000
1,α: Weight on other's payoff when behind,0.193,0.019,0.0,0.061,0.009,0.000,-0.328,0.073,0.000
2,β: Weight on other's payoff when ahead,0.494,0.020,0.0,0.095,0.012,0.000,-0.048,0.053,0.364
3,γ: Measure of positive reciprocity,0.099,0.024,0.0,-0.005,0.006,0.469,-0.028,0.030,0.364
4,δ: Measure of negative reciprocity,-0.081,0.018,0.0,-0.019,0.007,0.004,-0.015,0.035,0.669
5,σ: Choice sensitivity,0.019,0.001,0.0,0.049,0.004,0.000,0.015,0.002,0.000


Number of observations: 18,720
Number of subjects: 160
Log likelihood: -3,166.32


In [ ]:
# Create a single table with all the results and save it as a csv file in output

table2  = pd.DataFrame({"Parameters":param_names,"Str_altr_est_s1":[results_sm1[15],*results_sm1[0:5]],"Str_altr_se_s1":np.round([se_sm1[15],*se_sm1[0:5]],3),
                        "pval_str_s1":np.round([pvalues_s1[15],*pvalues_s1[0:5]],3),"Mod_altr_est_s1":[results_sm1[17],*results_sm1[10:15]],
                        "Mod_altr_se_s1":np.round([se_sm1[17],*se_sm1[10:15]],3),"pval_mod_s1":np.round([pvalues_s1[17],*pvalues_s1[10:15]],3),
                        "Beh_aver_est_s1":[results_sm1[16],*results_sm1[5:10]],"Beh_aver_se_s1":np.round([se_sm1[16],*se_sm1[5:10]],3),
                        "pval_beh_s1":np.round([pvalues_s1[16],*pvalues_s1[5:10]],3),
                        "Str_altr_est_s2":[results_sm2[15],*results_sm2[0:5]],"Str_altr_se_s2":np.round([se_sm2[15],*se_sm2[0:5]],3),
                        "pval_str_s2":np.round([pvalues_s2[15],*pvalues_s2[0:5]],3),"Mod_altr_est_s2":[results_sm2[17],*results_sm2[10:15]],
                        "Mod_altr_se_s2":np.round([se_sm2[17],*se_sm2[10:15]],3),"pval_mod_s2":np.round([pvalues_s2[17],*pvalues_s2[10:15]],3),
                        "Beh_aver_est_s2":[results_sm2[16],*results_sm2[5:10]],"Beh_aver_se_s2":np.round([se_sm2[16],*se_sm2[5:10]],3),
                        "pval_beh_s2":np.round([pvalues_s2[16],*pvalues_s2[5:10]],3)})

table2.to_csv('../output/table2_python.csv')